# Managing permissions

This example requires the Pro-feature to be active

In [1]:
import geoengine as ge

## Initialize Geo Engine as Admin

In [2]:
admin_client = ge.create_client("http://localhost:3030/api", ("admin@localhost", "adminadmin"))

## Create layer

In [3]:
root_collection = admin_client.layer_collection()
root_of_layerdb = admin_client.layer_listing_resolve(root_collection.items[1])
layer_id = admin_client.layer_collection_add_layer(
    root_of_layerdb,
    name="ports clone",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {
                        "data": "ne_10m_ports",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {
                        "data": "germany_outline",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                }
            }
        }
    },
    symbology=None,
)

root_of_layerdb = admin_client.layer_collection_reload(root_of_layerdb)
root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: b2fa607f-3620-4436-a349-b6015248b169
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Login anonymously, the layer is not visible

In [4]:
anon_client = ge.create_client("http://localhost:3030/api")
root_collection = anon_client.layer_collection()
root_of_layerdb = admin_client.layer_listing_resolve(root_collection.items[1])

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: b2fa607f-3620-4436-a349-b6015248b169
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Share layer with all anonymous users

In [5]:
resource = ge.Resource.from_layer_id(layer_id)
admin_client.permission_add(ge.ANONYMOUS_USER_ROLE_ID, resource, ge.Permission.READ)


{"roleId": "fd8e87bf-515c-4f36-8da6-1a53702ff102", "resource": {"type": "layer", "id": "b2fa607f-3620-4436-a349-b6015248b169"}, "permission": "Read"}


## Layer is now visible for anonymous users

In [6]:
root_collection = anon_client.layer_collection()
root_of_layerdb = anon_client.layer_listing_resolve(root_collection.items[1])

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone
description: test description
id: b2fa607f-3620-4436-a349-b6015248b169
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Create a user and a role

In [7]:
import requests as re
email = "foo@example.com"
password = "secret12345"

# register a user
response = re.post("http://localhost:3030/api/user", json={"email": email, "password": password, "realName": "Foo Bar"})
user_id = ge.UserId.from_response(response.json())

role_id = admin_client.role_add("Test Role")


{'id': '32e78a40-6df5-44b9-8808-1361395886a8'}


## Create a layer and share it with new role

In [8]:
root_collection = admin_client.layer_collection()
root_of_layerdb = admin_client.layer_listing_resolve(root_collection.items[1])

layer_id = admin_client.layer_collection_add_layer(
    root_of_layerdb,
    name="ports clone for new role",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {
                        "data": "ne_10m_ports",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {
                        "data": "germany_outline",
                        "attributeProjection": None,
                        "attributeFilters": None
                    }
                }
            }
        }
    },
    symbology=None,
)

resource = ge.Resource.from_layer_id(layer_id)
admin_client.permission_add(role_id, resource, ge.Permission.READ)


{"roleId": "8fe66f26-80a0-42ac-99a9-94780cff7070", "resource": {"type": "layer", "id": "beb48c88-04b3-4f02-8604-ce2aa7a27133"}, "permission": "Read"}


## Login as new user, the layer is not visible

In [9]:
new_user_client = ge.create_client("http://localhost:3030/api", (email, password))

root_collection = new_user_client.layer_collection() 
root_of_layerdb = new_user_client.layer_listing_resolve(root_collection.items[1])

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

## Assign role to user, layer is now visible

In [10]:
admin_client.role_assign(role_id, user_id)

root_collection = new_user_client.layer_collection()
root_of_layerdb = new_user_client.layer_listing_resolve(root_collection.items[1])

root_of_layerdb

Layer Collection
name: Layers
description: All available Geo Engine layers
id: 05102bb3-a855-4a37-8a8a-30026a91fef1
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74
items: Layer Collection
name: An empty collection
description: There is nothing here
id: a29f77cc-51ce-466b-86ef-d0ab2170bc0a
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: A test collection
description: Some layers for testing and an empty subcollection
id: 272bf675-2e27-4412-824c-287c1e6841ac
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer Collection
name: Unsorted
description: Unsorted Layers
id: ffb2dd9e-f5ad-427c-b7f1-c9a0c7a0ae3f
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74

       Layer
name: ports clone for new role
description: test description
id: beb48c88-04b3-4f02-8604-ce2aa7a27133
provider id: ce5e84db-cbf9-48a2-9a32-d4b7cc56ea74